In [ ]:
import os
import shutil
import flopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# ! Custom modules
from utils import pht3d_fsp
from utils.tools import calculate_MAS_sums, clear_directory
from utils.pht3d_utils import copy_pht3d_datab, run_pht3d_program

# clear_directory()

In [ ]:
date_series = pd.to_datetime([
    "2021/8/29",  "2021/9/10",  "2021/9/20",  "2021/10/1", "2021/10/11", "2021/11/21", "2021/12/2", 
    "2021/12/11", "2021/12/20", "2021/12/28", "2022/1/9",  "2022/2/21",  "2022/3/10",  "2022/3/20",
    "2022/3/30",  "2022/4/10",  "2022/5/1",   "2022/5/12", "2022/5/23",  "2022/6/22",  "2022/6/24",
    "2022/6/26",  "2022/6/28",  "2022/7/10",  "2022/7/21", "2022/8/1",   "2022/8/11"], format='%Y/%m/%d')
days = (date_series - date_series[0]).days.to_numpy() # ! 计算每个日期与第一个日期的差值

time_deltas = date_series.diff().dropna().days.tolist() # ! 计算相邻日期之间的差值
time_deltas = np.array(time_deltas)

In [ ]:
days

In [ ]:
data_folder = 'case_qarhan'
model_name = 'case_qarhan_modflow'
model_ws = './models_folder/' + data_folder

Lx = 10.0
Ly = 1560.0
Lz = 2.0

nrow = 1
ncol = 15
nlay = 1

delr = Lx / ncol
delc = 10.0
top = 2700.0
botm = 2698.0

hk = 100 # m/d

nper = time_deltas.shape[0]
perlen = time_deltas
nstp = np.full(time_deltas.shape, 1)# time_deltas//5
steady = np.full(time_deltas.shape, False)

model = flopy.modflow.Modflow(modelname=model_name, model_ws=model_ws, exe_name='./bin/mf2005', verbose=0)
dis = flopy.modflow.ModflowDis(model, nlay=nlay, nrow=nrow, ncol=ncol, 
                              delr=delr, delc=delc, top=top, botm=botm,
                              nper=nper, perlen=perlen, nstp=nstp, steady=steady)

ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[0, 0, ncol-1] = -1
ibound[0, 0, 0] = -1
strt = np.ones((nlay, nrow, ncol), dtype=np.float32) * 2699.0
bas = flopy.modflow.ModflowBas(model, ibound=ibound, strt=strt)

lpf = flopy.modflow.ModflowLpf(model, hk=hk, laytyp=0, ipakcb=53)

chd_data = {0: [[0, 0, ncol-1, 2700.0, 2700.0], [0, 0, 0, 2699.0, 2699.0]]}
chd = flopy.modflow.ModflowChd(model, stress_period_data=chd_data, extension="chd1")

wel_data = {0: [[0, 0, 5, -1200.0],]}
wel = flopy.modflow.ModflowWel(model, stress_period_data=wel_data)

lmt = flopy.modflow.ModflowLmt(model, output_file_name='mt3d_link.ftl')

stress_period_data = {}
for kper in range(nper):
    for kstp in range(nstp[kper]):
        stress_period_data[(kper, kstp)] = ["save head", "save drawdown", "save budget", "print head", "print budget",]
oc = flopy.modflow.ModflowOc(model, stress_period_data=stress_period_data, compact=True)
pcg = flopy.modflow.ModflowPcg(model=model)

model.write_input()
success, mfoutput = model.run_model(silent=True, pause=False, report=True)
if not success:
    raise Exception('MODFLOW did not terminate normally.')

In [ ]:
head = flopy.utils.binaryfile.HeadFile(model_ws+'/'+model_name+".hds").get_alldata()
from matplotlib.ticker import ScalarFormatter
fig, ax = plt.subplots()
ax.plot(head[0, 0, 0])
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
plt.title("head")
head.shape

In [ ]:

# ! 定义摩尔质量 (g/mol)
molecular_weights = {'HCO3': 61.0168, 'Cl': 35.453, 'K': 39.0983, 'Na': 22.989769, 'Ca': 40.078, 'Mg': 24.305,
                      'SO4': 96.06,   'Li': 6.94,  'Sr': 87.62,    'B': 10.81,     'Br': 79.904, 'NO3': 62.0049}
df = pd.read_csv("./data/case_qarhan/samples.csv")
# ! 定义一个函数进行单位转换
def mgL_to_molL(mgL, molecular_weight):
    return mgL / molecular_weight / 1000  # mg/L to mol/L
# ! 进行单位转换
for element, mw in molecular_weights.items():
    df[element] = df[element].apply(lambda x: mgL_to_molL(x, mw))
# ! 保存为新的CSV文件
# df.to_csv('converted_samples.csv', index=False)
df

In [ ]:
## Create PHT3D Reactive Transport Model, supported by PHT3D-FSP
## Define "spec", calling PHT3D-FSP (adopt function variables as needed)
spec = pht3d_fsp.create(
    xlsx_path="./", 
    xlsx_name=f"./data/{data_folder}/pht3d_species.xlsx",
    nlay=nlay, 
    nrow=nrow,
    ncol=ncol,
    ph_os=2,
    ph_temp=10,
    ph_asbin=1,
    ph_eps_aqu=0,
    ph_ph=0,
    # ph_print=0,
    ph_cb_offset=0,
    # ph_surf_calc_type="-diffuse_layer",
    write_ph="yes"
)
# Move the pht3d_ph.dat file into the models_folder
source = 'pht3d_ph.dat'
destination = os.path.join(model_ws, 'pht3d_ph.dat')
shutil.move(source, destination)

In [ ]:
ssm_data = {}
itype = flopy.mt3d.Mt3dSsm.itype_dict()
print(itype)
# {'CHD': 1, 'BAS6': 1, 'PBC': 1, 'WEL': 2, 'DRN': 3, 'RIV': 4, 'GHB': 5, 'MAS': 15, 'CC': -1}

In [ ]:
# wel_spec={}
# for i in range(1):
#     for key in spec.keys():
#         wel_spec[key]=spec[key][0,0,0]

#     wel_spec['k'] = 0.363135993
#     wel_spec['na'] = 1.88475143
#     wel_spec['ca'] = 0.045785718
#     wel_spec['mg'] = 2.15334293355276
#     wel_spec['c4'] = 0.0125998085773098
#     wel_spec['s6'] = 0.046241932
#     wel_spec['cl'] = 6.08278565988773
#     wel_spec['ph'] = 6.46
#     wel_spec['pe'] = 4.0
#     wel_spec['carnallite'] = 0
#     wel_spec['polyhalite'] = 0
#     wel_spec['gypsum'] = 0
#     wel_spec['halite'] = 0
#     wel_spec['sylvite'] = 0
#     wel_spec['calcite'] = 0
#     wel_spec['dolomite'] = 0

#     ssm_per=[0, 0, 0, 0.0, itype['CHD']]
#     for key in spec.keys():
#         ssm_per.append(wel_spec[key])
#     ssm_data[i] = ssm_per

# ssm_data

In [ ]:
ssm_data = {
    0: [
        #                                 k      na     ca     mg     c4     s6     cl     ph     pe carnallite polyhalite gypsum halite sylvite calcite dolomite
        [0, 0, 0,      0.0, itype['CHD'], 0.363, 3.884, 0.012, 0.045, 0.015, 0.049, 6.082, 6.460, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.035, 3.739, 0.039, 1.521, 0.013, 0.066, 5.634, 6.940, 5.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    1: [
        [0, 0, 0,      0.0, itype['CHD'], 0.511, 3.415, 0.041, 2.208, 0.012, 0.039, 6.332, 6.450, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.045, 3.188, 0.032, 1.568, 0.013, 0.059, 5.303, 6.990, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    2: [
        [0, 0, 0,      0.0, itype['CHD'], 0.428, 3.296, 0.039, 2.234, 0.013, 0.044, 6.152, 6.560, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.043, 3.143, 0.033, 1.578, 0.013, 0.066, 5.247, 7.080, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    3: [
        [0, 0, 0,      0.0, itype['CHD'], 0.326, 3.703, 0.032, 2.120, 0.014, 0.041, 6.238, 6.580, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.049, 3.179, 0.034, 1.519, 0.013, 0.066, 5.197, 6.890, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    4: [
        [0, 0, 0,      0.0, itype['CHD'], 0.249, 3.687, 0.029, 2.033, 0.014, 0.044, 5.958, 6.660, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.036, 3.267, 0.028, 1.518, 0.013, 0.058, 5.269, 7.030, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
        1: [
        [0, 0, 0,      0.0, itype['CHD'], 0.156, 3.305, 0.030, 1.647, 0.013, 0.051, 5.700, 6.880, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.049, 3.093, 0.033, 1.477, 0.013, 0.061, 5.028, 7.170, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    5: [
        [0, 0, 0,      0.0, itype['CHD'], 0.051, 3.905, 0.032, 1.339, 0.012, 0.053, 4.581, 7.210, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.043, 3.792, 0.031, 1.405, 0.013, 0.058, 4.581, 7.120, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    6: [
        [0, 0, 0,      0.0, itype['CHD'], 0.043, 3.955, 0.030, 1.321, 0.000, 0.059, 4.751, 7.380, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.041, 3.860, 0.031, 1.370, 0.000, 0.061, 4.744, 7.410, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],   
    7: [
        [0, 0, 0,      0.0, itype['CHD'], 0.049, 3.432, 0.031, 1.573, 0.000, 0.069, 5.650, 7.300, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.043, 4.207, 0.031, 1.517, 0.000, 0.066, 5.270, 7.360, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    8: [
        [0, 0, 0,      0.0, itype['CHD'], 0.047, 4.308, 0.032, 1.567, 0.000, 0.071, 5.636, 7.250, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.044, 4.198, 0.032, 1.531, 0.000, 0.075, 5.563, 7.300, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
    9: [
        [0, 0, 0,      0.0, itype['CHD'], 0.038, 4.615, 0.030, 1.020, 0.012, 0.051, 3.633, 7.230, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.038, 4.663, 0.031, 1.367, 0.012, 0.053, 4.369, 7.080, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
       #                                   k      na     ca     mg     c4     s6     cl     ph     pe carnallite polyhalite gypsum halite sylvite calcite dolomite
   10: [
        [0, 0, 0,      0.0, itype['CHD'], 0.066, 4.317, 0.033, 1.172, 0.012, 0.052, 4.667, 7.060, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第一个数据 BS
        [0, 0, ncol-1, 0.0, itype['CHD'], 0.038, 1.597, 0.031, 1.180, 0.012, 0.054, 3.928, 7.220, 4.000, 0, 0, 0, 0, 0, 0, 0, 0], # 第二个数据 HS
       ],
}
ssm_data

In [ ]:
# # delete single -Formula
# input_file = model_ws+'/pht3d_ph.dat'

# with open(input_file, 'r') as file:
#     lines = file.readlines()
# lines = [line for line in lines if line.strip() != '-Formula']
# with open(input_file, 'w') as file:
#     file.writelines(lines)

In [ ]:
model_name = 'case_qarhan_mt3dms'
icbund = np.abs(ibound)

mt = flopy.mt3d.Mt3dms(model_name, model_ws=model_ws, exe_name='./bin/mt3dms',
                      ftlfilename='mt3d_link.ftl', modflowmodel=model, namefile_ext='nam_pht3d')

prsity = 0.32
save_times = np.linspace(0, 347, 346)

exec(f'btn = flopy.mt3d.Mt3dBtn(mt, \
                                nper=nper, perlen=perlen, nstp=nstp,\
                                nlay=nlay, ncol=ncol, nrow=nrow, \
                                laycon=[1], prsity=prsity, icbund=icbund, nprs=1, mxstrn=50000, tsmult=1, timprs=save_times, \
    ncomp={pht3d_fsp.create.ncomp}, mcomp={pht3d_fsp.create.mcomp}, {pht3d_fsp.create.sconc_btn})') # call BTN package in this way to invoke PHT3D-FSP variables

adv = flopy.mt3d.Mt3dAdv(mt, mixelm=-1, percel=0.75, mxpart=5000, nadvfd=0)
dsp = flopy.mt3d.Mt3dDsp(mt, al=0.0067, trpt=0.1, trpv=0.1, multiDiff=True, dmcoef=0.0,)
gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=1, iter1=50, isolve=3, cclose=0.01, iprgcg=0)
exec(f'ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=ssm_data, mxss=1000)')
mt.write_input()

In [ ]:
# Manually add the PHC reaction package to the nam file and push out as pht3d.nam
s='PHC               39  pht3d_ph.dat\n'
namfiletxt=open(model_ws + '/'+ mt.namefile, 'r').read()
pht3d_nam = namfiletxt+s
file = open(model_ws + '/pht3d.nam','w')
file.write(pht3d_nam)
file.close()

In [ ]:
copy_pht3d_datab(source_folder=data_folder)

run_pht3d_program(work_dir=data_folder)

In [ ]:
days

In [ ]:
results_k  = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D001.UCN').get_alldata()
results_na = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D002.UCN').get_alldata()
results_ca = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D003.UCN').get_alldata()
results_mg = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D004.UCN').get_alldata()
results_c4 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D005.UCN').get_alldata()
results_s6 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D006.UCN').get_alldata()
results_cl = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D007.UCN').get_alldata()
results_ph = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D008.UCN').get_alldata()
results_pe = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D009.UCN').get_alldata()
results_carnallite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D011.UCN').get_alldata()
results_polyhalite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D012.UCN').get_alldata()
results_gypsum = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D013.UCN').get_alldata()
results_halite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D014.UCN').get_alldata()
results_sylvite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D015.UCN').get_alldata()
results_calcite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D016.UCN').get_alldata()
results_dolomite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D017.UCN').get_alldata()

# days = np.linspace(0, 120, 120).astype(int)
location = 5
inter_times = 340
fig = plt.figure(figsize=(14, 6))
ax = fig.add_subplot(3, 6, 1)
ax.plot(days, results_k[days, 0, 0, location], color="red",)
ax.set_title("K")
ax = fig.add_subplot(3, 6, 2)
ax.plot(days, results_na[days, 0, 0, location], color="red")
ax.set_title("Na")
ax = fig.add_subplot(3, 6, 3)
ax.plot(days, results_c4[days, 0, 0, location], color="red")
ax.set_title("CO3")
ax = fig.add_subplot(3, 6, 4)
ax.plot(days, results_ca[days, 0, 0, location], color="red")
ax.set_title("Ca")
ax = fig.add_subplot(3, 6, 5)
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.plot(days, results_mg[days, 0, 0, location], color="red")
ax.set_title("Mg")
ax = fig.add_subplot(3, 6, 6)
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.plot(days, results_s6[days, 0, 0, location], color="red")
ax.set_title("SO4")
ax = fig.add_subplot(3, 6, 7)
ax.plot(days, results_cl[days, 0, 0, location], color="red")
ax.set_title("Cl")
ax = fig.add_subplot(3, 6, 8)
ax.plot(days, results_ph[days, 0, 0, location], color="red")
ax.set_title("pH")
ax = fig.add_subplot(3, 6, 9)
ax.plot(days, results_pe[days, 0, 0, location], color="red")
ax.set_title("pe")
ax = fig.add_subplot(3, 6, 10)
ax.plot(days, results_carnallite[days, 0, 0, location], color="red")
ax.set_title(f"Carnallite\nKCl·MgCl2·6H2O")
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax = fig.add_subplot(3, 6, 11)
ax.plot(days, results_polyhalite[days, 0, 0, location], color="red")
ax.set_title(f"Polyhalite\nK2Ca2Mg[SO4]4·2H2O")
ax = fig.add_subplot(3, 6, 12)
ax.plot(days, results_gypsum[days, 0, 0, location], color="red")
ax.set_title(f"Gypsum\nCaSO4")
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax = fig.add_subplot(3, 6, 13)
ax.plot(days, results_halite[days, 0, 0, location], color="red")
ax.set_title("Halite NaCl")
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax = fig.add_subplot(3, 6, 14)
ax.plot(days, results_sylvite[days, 0, 0, location], color="red")
ax.set_title("Sylvite KCl")
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax = fig.add_subplot(3, 6, 15)
ax.plot(days, results_calcite[days, 0, 0, location], color="red")
ax.set_title("Calcite CaCO3")
ax = fig.add_subplot(3, 6, 16)
ax.plot(days, results_dolomite[days, 0, 0, location], color="red")
ax.set_title(f"Dolomite\nCaMg(CO3)2")
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
plt.subplots_adjust(top=None, bottom=None, left=None, right=None, hspace=0.8, wspace=0.5)

In [ ]:
results_k  = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D001.UCN').get_alldata()
results_na = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D002.UCN').get_alldata()
results_ca = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D003.UCN').get_alldata()
results_mg = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D004.UCN').get_alldata()
results_c4 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D005.UCN').get_alldata()
results_s6 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D006.UCN').get_alldata()
results_cl = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D007.UCN').get_alldata()
results_ph = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D008.UCN').get_alldata()
results_pe = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D009.UCN').get_alldata()
results_carnallite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D010.UCN').get_alldata()
results_polyhalite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D011.UCN').get_alldata()
results_gypsum = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D012.UCN').get_alldata()
results_halite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D013.UCN').get_alldata()
results_sylvite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D014.UCN').get_alldata()
results_calcite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D015.UCN').get_alldata()
results_dolomite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D016.UCN').get_alldata()

location = 5
inter_times = 340
fig = plt.figure(figsize=(14, 6))
ax = fig.add_subplot(3, 6, 1)
ax.plot(results_k[56, 0, 0, :], color="red", linestyle='None', marker='o', markersize=1)
ax.set_title("K")
ax = fig.add_subplot(3, 6, 2)
ax.plot(results_na[56, 0, 0, :], color="red")
ax.set_title("Na")
ax = fig.add_subplot(3, 6, 3)
ax.plot(results_c4[56, 0, 0, :], color="red")
ax.set_title("CO3")
ax = fig.add_subplot(3, 6, 4)
ax.plot(results_ca[56, 0, 0, :], color="red")
ax.set_title("Ca")
ax = fig.add_subplot(3, 6, 5)
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.plot(results_mg[56, 0, 0, :], color="red")
ax.set_title("Mg")
ax = fig.add_subplot(3, 6, 6)
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.plot(results_s6[56, 0, 0, :], color="red")
ax.set_title("SO4")
ax = fig.add_subplot(3, 6, 7)
ax.plot(results_cl[56, 0, 0, :], color="red")
ax.set_title("Cl")
ax = fig.add_subplot(3, 6, 8)
ax.plot(results_ph[200, 0, 0, :], color="red")
ax.set_title("pH")
ax = fig.add_subplot(3, 6, 9)
ax.plot(results_pe[:inter_times, 0, 0, location], color="red")
ax.set_title("pe")
ax = fig.add_subplot(3, 6, 10)
ax.plot(results_carnallite[1, 0, 0, :], color="red")
ax.set_title(f"Carnallite\nKCl·MgCl2·6H2O")
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax = fig.add_subplot(3, 6, 11)
ax.plot(results_polyhalite[-1, 0, 0, :], color="red")
ax.set_title(f"Polyhalite\nK2Ca2Mg[SO4]4·2H2O")
ax = fig.add_subplot(3, 6, 12)
ax.plot(results_gypsum[-1, 0, 0, :], color="red", linestyle='None', marker='o', markersize=1)
ax.set_title(f"Gypsum\nCaSO4")
ax = fig.add_subplot(3, 6, 13)
ax.plot(results_halite[30, 0, 0, :], color="red")
ax.set_title("Halite NaCl")
ax = fig.add_subplot(3, 6, 14)
ax.plot(results_sylvite[-1, 0, 0, :], color="red")
ax.set_title("Sylvite KCl")
ax = fig.add_subplot(3, 6, 15)
ax.plot(results_calcite[:inter_times, 0, 0, location], color="red")
ax.set_title("Calcite CaCO3")
ax = fig.add_subplot(3, 6, 16)
ax.plot(results_dolomite[16, 0, 0, :], color="red")
ax.set_title(f"Dolomite\nCaMg(CO3)2")
plt.subplots_adjust(top=None, bottom=None, left=None, right=None, hspace=0.8, wspace=0.5)

In [ ]:
(2.2246 * (10 ** (-10)) * ((815.81 - 3.4390) ** (3.8312))) / 8640000

In [ ]:
# results_k = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D001.UCN').get_alldata()
# results_na = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D002.UCN').get_alldata()
# results_ca = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D003.UCN').get_alldata()
# results_mg = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D004.UCN').get_alldata()
# results_c4 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D005.UCN').get_alldata()
# results_s6 = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D006.UCN').get_alldata()
# results_cl = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D007.UCN').get_alldata()
# results_ph = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D008.UCN').get_alldata()
# results_pe = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D009.UCN').get_alldata()
# results_carnallite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D010.UCN').get_alldata()
# results_polyhalite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D011.UCN').get_alldata()
# results_gypsum = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D012.UCN').get_alldata()
# results_halite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D013.UCN').get_alldata()
# results_sylvite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D014.UCN').get_alldata()
# results_calcite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D015.UCN').get_alldata()
# results_dolomite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D016.UCN').get_alldata()

# time_ = 1
# fig = plt.figure(figsize=(14, 6))
# ax = fig.add_subplot(3, 6, 1)
# ax.plot(results_k[time_, 0, 0, :], color="red")
# ax.set_title("K")
# ax = fig.add_subplot(3, 6, 2)
# ax.plot(results_na[time_, 0, 0, :], color="red")
# ax.set_title("Na")
# ax = fig.add_subplot(3, 6, 3)
# ax.plot(results_c4[time_, 0, 0, :], color="red")
# ax.set_title("CO3")
# ax = fig.add_subplot(3, 6, 4)
# ax.plot(results_ca[time_, 0, 0, :], color="red")
# ax.set_title("Ca")
# ax = fig.add_subplot(3, 6, 5)
# ax.plot(results_mg[time_, 0, 0, :], color="red")
# ax.set_title("Mg")
# ax = fig.add_subplot(3, 6, 6)
# ax.plot(results_s6[time_, 0, 0, :], color="red")
# ax.set_title("SO4")
# ax = fig.add_subplot(3, 6, 7)
# ax.plot(results_cl[time_, 0, 0, :], color="red")
# ax.set_title("Cl")
# ax = fig.add_subplot(3, 6, 8)
# ax.plot(results_ph[time_, 0, 0, :], color="red")
# ax.set_title("pH")
# ax = fig.add_subplot(3, 6, 9)
# ax.plot(results_pe[time_, 0, 0, :], color="red")
# ax.set_title("pe")
# ax = fig.add_subplot(3, 6, 10)
# ax.plot(results_carnallite[time_, 0, 0, :], color="red")
# ax.set_title(f"Carnallite\nKCl·MgCl2·6H2O")
# ax = fig.add_subplot(3, 6, 11)
# ax.plot(results_polyhalite[time_, 0, 0, :], color="red")
# ax.set_title(f"Polyhalite\nK2Ca2Mg[SO4]4·2H2O")
# ax = fig.add_subplot(3, 6, 12)
# ax.plot(results_gypsum[time_, 0, 0, :], color="red")
# ax.set_title("Gypsum CaSO4")
# ax = fig.add_subplot(3, 6, 13)
# ax.plot(results_halite[time_, 0, 0, :], color="red")
# ax.set_title("Halite NaCl")
# ax = fig.add_subplot(3, 6, 14)
# ax.plot(results_sylvite[time_, 0, 0, :], color="red")
# ax.set_title("Sylvite KCl")
# ax = fig.add_subplot(3, 6, 15)
# ax.plot(results_calcite[time_, 0, 0, :], color="red")
# ax.set_title("Calcite CaCO3")
# ax = fig.add_subplot(3, 6, 16)
# ax.plot(results_dolomite[time_, 0, 0, :], color="red")
# ax.set_title("Dolomite CaMg(CO3)2")
# plt.subplots_adjust(top=None, bottom=None, left=None, right=None, hspace=0.8, wspace=0.5)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 定义公式函数
def calculate_r(C):
    return 0.0044 * (0.2066 - C)**0.7

# 定义C的范围
C_values = np.linspace(0, 0.2066, 500)
r_values = calculate_r(C_values)

# 绘制图形
plt.figure(figsize=(10, 6))
plt.plot(C_values, r_values, label='r = 0.0044 * (C - 0.2066)^0.7')
plt.xlabel('C')
plt.ylabel('r')
plt.title('r vs C')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# results_k = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D001.UCN').get_alldata()[2, 0, 0]*1000
# results_mg = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D002.UCN').get_alldata()[2, 0,0]*1000
# results_cl = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D003.UCN').get_alldata()[2, 0,0]*1000
# results_ph = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D004.UCN').get_alldata()[2, 0,0]
# results_calcite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D010.UCN').get_alldata()[0,  0,0]
# results_dolomite = flopy.utils.binaryfile.UcnFile(model_ws+'/PHT3D011.UCN').get_alldata()[1, 0,0]

# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(3, 2, 1)
# ax.plot(results_cl, color="red")
# ax.set_title("Cl")
# # ax.set_ylim(0, 2.0)
# ax = fig.add_subplot(3, 2, 2)
# ax.plot(results_ph, color="red")
# ax.set_title("pH")
# # ax.set_ylim(7.0, 10.0)
# ax = fig.add_subplot(3, 2, 3)
# ax.plot(results_mg, color="red")
# ax.set_title("Mg")
# # ax.set_ylim(0.0, 1.2)
# ax = fig.add_subplot(3, 2, 4)
# ax.plot(results_ca, color="red")
# ax.set_title("Ca")
# # ax.set_ylim(0.0, 0.30)
# ax = fig.add_subplot(3, 2, 5)
# ax.plot(results_calcite, color="red")
# ax.set_title("Calcite")
# # ax.set_ylim(0.0, 0.025)
# ax = fig.add_subplot(3, 2, 6)
# ax.plot(results_dolomite, color="red")
# ax.set_title("Dolomite")
# # ax.set_ylim(0.0, 0.014)

# plt.subplots_adjust(top=None, bottom=None, left=None, right=None, hspace=0.4, wspace=None)